## Stints Dashboard: Tyre Compound Usage by Driver

This dashboard visualizes each driver's tyre compound usage and stint durations across a session. Select a session and driver to see their compound changes and stint progression.

In [ ]:
# Import required libraries
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

In [ ]:
# Load stints and session metadata
stints = pd.read_csv('../data/cleaned/stints.csv')
sessions = pd.read_csv('../data/cleaned/sessions.csv')
# Ensure date column is datetime and sort by date ascending
if 'date' in sessions.columns:
    sessions['date'] = pd.to_datetime(sessions['date'], errors='coerce')
    sessions = sessions.sort_values('date', ascending=True)
# Get session_keys in date order
ordered_session_keys = sessions['session_key'].tolist()

# Join on session_key
stints_sessions = stints.merge(sessions, on='session_key', how='left', suffixes=('', '_session'))

# Create new session_id: <session_key>-<session_name>-<circuit_short_name>
if 'session_name' in stints_sessions.columns and 'circuit_short_name' in stints_sessions.columns:
    stints_sessions['session_id'] = stints_sessions['session_key'].astype(str) + '-' + \
        stints_sessions['session_name'].astype(str) + '-' + \
        stints_sessions['circuit_short_name'].astype(str)
else:
    stints_sessions['session_id'] = stints_sessions['session_key'].astype(str)

# Prepare dropdown options for session_id in date order
stint_session_id_ordered = []
for sk in ordered_session_keys:
    subset = stints_sessions[stints_sessions['session_key'] == sk]
    if not subset.empty:
        session_id = subset['session_id'].iloc[0]
        if session_id not in stint_session_id_ordered:
            stint_session_id_ordered.append(session_id)
stint_session_options = [
    {"label": sid, "value": sid} for sid in stint_session_id_ordered
]

# Aggregate stats for each session_id and compound
stints_agg = stints_sessions.groupby(['session_id', 'compound']).agg({
    'lap_start': 'min',
    'lap_end': 'max',
    'stint_number': 'count',
    'tyre_age_at_start': ['mean', 'min', 'max']
}).reset_index()
stints_agg.columns = ['_'.join(col).strip('_') for col in stints_agg.columns.values]

In [ ]:
# Stints dashboard app with session and driver filters
app = Dash(__name__)

# Prepare driver options for dropdown with unique key: driver_number-last_name
fixed_drivers = {
    1: 'Verstappen',
    81: 'Piastri',
    16: 'Leclerc',
    4: 'Norris',
    55: 'Sainz',
}
driver_options = [
    {"label": f"{num} - {name}", "value": f"{num}-{name}"} for num, name in fixed_drivers.items()
]

app.layout = html.Div([
    html.H1('F1 Stints Dashboard: Tyre Compound Usage by Driver'),
    dcc.Dropdown(
        id='stint-session-dropdown',
        options=stint_session_options,
        value=stints_sessions['session_id'].unique()[0],
        clearable=False,
        style={'width': '60%'}
    ),
    dcc.Dropdown(
        id='driver-dropdown',
        options=driver_options,
        value=driver_options[0]['value'],
        clearable=False,
        style={'width': '40%', 'marginTop': 10, 'marginBottom': 10}
    ),
    html.Div(id='stints-driver-table', style={'width': '60%', 'margin': 'auto', 'marginBottom': 20}),
    dcc.Graph(id='driver-stints-compound-graph'),
])

@app.callback(
    Output('driver-stints-compound-graph', 'figure'),
    Output('stints-driver-table', 'children'),
    Input('stint-session-dropdown', 'value'),
    Input('driver-dropdown', 'value')
)
def update_driver_stints_dashboard(selected_session_id, selected_driver_key):
    # Parse driver_number from key
    driver_number = int(selected_driver_key.split('-')[0])
    filtered = stints_sessions[(stints_sessions['session_id'] == selected_session_id) & (stints_sessions['driver_number'] == driver_number)]
    # Define a fixed color map for compounds
    compound_palette = {
        'SOFT': '#e41a1c',
        'MEDIUM': '#ff7f00',
        'HARD': '#377eb8',
        'INTERMEDIATE': '#4daf4a',
        'WET': '#984ea3',
    }
    # Get only the compounds present for this driver/session
    compounds_present = filtered['compound'].unique()
    color_discrete_map = {c: compound_palette.get(c, '#999999') for c in compounds_present}
    # Plot each stint as a line colored by compound
    fig = px.line(
        filtered,
        x='lap_start',
        y='compound',
        color='compound',
        line_group='stint_number',
        markers=True,
        title=f'Driver {selected_driver_key} Tyre Compound Usage ({selected_session_id})',
        labels={'lap_start': 'Lap', 'compound': 'Tyre Compound'},
        color_discrete_map=color_discrete_map
    )
    # Add duration as horizontal segments for each stint, using locked color
    for _, row in filtered.iterrows():
        fig.add_scatter(x=[row['lap_start'], row['lap_end']], y=[row['compound'], row['compound']],
                        mode='lines', line=dict(width=8, color=color_discrete_map.get(row['compound'], '#999999')),
                        name=row['compound'],
                        showlegend=False)
    # Table: stint number, lap_start, lap_end, compound, tyre_age_at_start
    if not filtered.empty:
        stats_table = html.Table([
            html.Tr([html.Th(col.replace('_', ' ').title()) for col in ['stint_number', 'lap_start', 'lap_end', 'compound', 'tyre_age_at_start']]),
            *[html.Tr([html.Td(row[c]) for c in ['stint_number', 'lap_start', 'lap_end', 'compound', 'tyre_age_at_start']]) for _, row in filtered.iterrows()]
        ], style={'margin': 'auto', 'border': '1px solid #ccc', 'borderCollapse': 'collapse', 'width': '100%'})
    else:
        stats_table = html.Div('No stint data available for this driver/session.')
    return fig, stats_table

app.run(jupyter_mode='inline')